In [24]:
import pandas as pd
import numpy as np

import re
from __future__ import division

In [25]:
im= pd.read_csv("./imdb.csv");
rt= pd.read_csv("./rotten_tomatoes.csv");
print('The imdb shape: %d x %d' % im.shape)
print('The rt shape: %d x %d' % im.shape)

The imdb shape: 6407 x 13
The rt shape: 6407 x 13


In [26]:
#Split words in the n-grams
def ngrams(string, n=2):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

    
#compute dice for array or string 
def dice_coefficient(a, b,n=2):
    """dice coefficient 2nt/na + nb."""
    
    #For comparing strings 
    if isinstance(a,str) & isinstance(b,str):
        a = ngrams(a,n)
        b = ngrams(b,n)
        
    #For comparing lists  
    a_bigrams = set(a)
    b_bigrams = set(b)
    overlap = len(a_bigrams & b_bigrams)
    return overlap * 2.0/(len(a_bigrams) + len(b_bigrams))

In [27]:
dice_coefficient("Name","Nameee")

0.8

In [28]:
rt.head(0)

,Id,Name,Year,Release Date,Director,Creator,Actors,Cast,Language,Country,Duration,RatingValue,RatingCount,ReviewCount,Genre,Filming Locations,Description


In [29]:
im.head(0)

,Id,Name,YearRange,ReleaseDate,Director,Creator,Cast,Duration,RatingValue,ContentRating,Genre,Url,Description



## General Truth

In [30]:
#Ibmd : RottenTomatoes
GT = {
      "Name":"Name",
      "ReleaseDate":"Release Date",
      "RatingValue":"RatingValue",
      "Director":"Director",
      "Creator":"Creator",
      "YearRange":"Year",
      "Genre":"Genre",
      "Duration":"Duration",
      "Cast":"Cast",
        "Description":"Description"
     }

## 1. Label Based Mapping

Here, we want to find the correspondences between the columns from the two datasets
with the help of only schema headers.
1. Provide an algorithm. Specify the input, output, similarity function, and time
complexity.
2. Implement the algorithm and report the results. Is there any parameter that affects
the results?
3. What is the upsides and downsides of this method? When does it work and when
not?

### Solution: 
*Dice-distance based algoritgm*

**get_mapping** is the main function of the algortihm. It takes two data tables (padans DataFrame) and a treshold value for the N-Grams computed using Dice Score.

In [117]:
def get_mapping(table_a,table_b,treshold=0.5,n=2):
    #print("Treshold: {}, {}-grams".format(treshold,n))
    A =[ str(a) for a in table_a.columns]
    B =[ str(b) for b in table_b.columns]
    #HardCoded skiping IDs
    A.remove('Id')
    B.remove('Id')
    #List of arrays containg header label and it's Ngram represatantion,
    rt_cl = [[ngrams(cl,n),cl]for cl in A]
    im_cl = [[ngrams(cl,n),cl] for cl in B]
    df = pd.DataFrame(columns=["tab_a_entry","tab_b_entry","coeff"])
    for r,r_ in rt_cl:
        for i,i_ in im_cl:
            dc = dice_coefficient(r,i)

            if (dc >= treshold ) or (r_ in i_):
                df = df.append(pd.Series([r_,i_,dc],index=["tab_a_entry","tab_b_entry","coeff"]),ignore_index=True)

    df = df.groupby(['tab_a_entry'], sort=True)['tab_a_entry','tab_b_entry','coeff'].max()
    return dict(zip(df["tab_a_entry"],df['tab_b_entry']))

### Evaluation Methods

In [118]:
def compute_recall(mapping):
    total_found =len(mapping.items())
    #print("Recall {}".format(float(total_found/len(GT.items()))))
    return float(total_found/len(GT.items()))
                 
def compute_precision(mapping):
    precision = set(mapping.items()) & set(GT.items())
    true_matches = len(precision)
    total_found =len(mapping.items())
    #print("Precision {}".format(float(true_matches/total_found)))
    return float(true_matches/total_found)

In [119]:
def what_is_missing_or_wrong(mapping):
    #Helper method
    def _removekey(d, key):
        r = dict(d)
        del r[key]
        return r
    
    missing = list()
    for x in GT.items():
        try:
            mapping= _removekey(mapping,x[0])
        except KeyError as e:
            #print("Missing : "+x[0])
            missing.append(x)
                               
    return missing, mapping

In [120]:
def test_algo(tableA,tableB,to_csv=False):
    treshold_test_axis = np.arange(0.001, 1, .02);
    ngram_test_axis = np.arange(1,5)

    df = pd.DataFrame(columns=["treshold","ngrams","precision","recall","total_matches","uniqie_matches"])
    for t in treshold_test_axis:
        for n in ngram_test_axis:
            print(t,n)
            mp =get_mapping(tableA,tableB,t,n)
            prec =compute_precision(mp)
            recall= compute_recall(mp)
            unique = set(mp.items())
            df = df.append(pd.Series([t,n,prec,recall,len(mp.items()),len(unique)],index=["treshold","ngrams","precision","recall","total_matches","uniqie_matches"]),ignore_index=True)
    df['precision_normalised']=(df['precision'] - df['precision'].mean()) / (df['precision'].max() - df['precision'].min())
    df['recall_normalised']=  (df['recall'] - df['recall'].mean()) / (df['recall'].max() - df['recall'].min())
    if to_csv:
        df.to_csv('test.csv')
    return df

In [ ]:
test_ = test_algo(im,rt,True)

(0.001, 1)
(0.001, 2)
(0.001, 3)
(0.001, 4)
(0.031, 1)
(0.031, 2)
(0.031, 3)
(0.031, 4)
(0.060999999999999999, 1)
(0.060999999999999999, 2)
(0.060999999999999999, 3)
(0.060999999999999999, 4)
(0.090999999999999998, 1)
(0.090999999999999998, 2)
(0.090999999999999998, 3)
(0.090999999999999998, 4)
(0.121, 1)
(0.121, 2)
(0.121, 3)
(0.121, 4)
(0.151, 1)
(0.151, 2)
(0.151, 3)
(0.151, 4)
(0.18099999999999999, 1)
(0.18099999999999999, 2)
(0.18099999999999999, 3)
(0.18099999999999999, 4)
(0.21099999999999999, 1)
(0.21099999999999999, 2)
(0.21099999999999999, 3)
(0.21099999999999999, 4)
(0.24099999999999999, 1)
(0.24099999999999999, 2)
(0.24099999999999999, 3)
(0.24099999999999999, 4)
(0.27100000000000002, 1)
(0.27100000000000002, 2)
(0.27100000000000002, 3)
(0.27100000000000002, 4)
(0.30099999999999999, 1)
(0.30099999999999999, 2)
(0.30099999999999999, 3)
(0.30099999999999999, 4)
(0.33099999999999996, 1)
(0.33099999999999996, 2)
(0.33099999999999996, 3)
(0.33099999999999996, 4)
(0.3609999999999

In [122]:
test_.to_csv("test.csv")

## Task 2
#### Instance-Based Schema Matching


In [334]:
im= pd.read_csv("./imdb.csv");
rt= pd.read_csv("./rotten_tomatoes.csv");

In [ ]:
im_cl=im.columns.values
rt_cl=rt.columns.values

In [ ]:
class Duplicate():
    def __init__(self,src,target):
        self.sCol = src.columns.values
        self.tCol = target.columns.values
        self.src=src.dropna()
        self.target=target.dropna()
        self.duplicates= list()
        
    def find_duplicate(self):
        example_tp =self.src.sample(1).values;
        for col in self.tCol:
            if !self.duplicates:
                self.target[col]